In [0]:
import time
from pyspark import *
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains, struct, collect_list, row_number

# Optimización con Persistencia: cache()



In [0]:
# Cargar datos MovieLens (ratings.csv) suponiendo que está en /dbfs/FileStore/movielens/ratings.csv
# Tabla Ratings
ratings_schema  = StructType(fields=[
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True),
    StructField("rating",DecimalType(precision=2,scale=1),True),
    StructField("timestamp",LongType(),True)
])
ratingsDf = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(ratings_schema)\
    .csv("/Volumes/big_data_ii_2025/spark_examples/spark_data/ratings_full.csv")\
    .withColumn(\
            "date",\
            from_unixtime("timestamp", "yyyyMMdd"))\
                .drop('timestamp')

In [0]:
%time
# Primera consulta: calcular promedio de calificación por película
start = time.time()
ratingsDf.groupBy("movieId").avg("rating").count()
print("Sin cache:", time.time() - start, "segundos")

CPU times: user 3 μs, sys: 1e+03 ns, total: 4 μs
Wall time: 5.48 μs
Sin cache: 4.341420412063599 segundos


In [0]:
%time
# Aplicar cache
ratingsDf.cache()
print("Tamaño: ", ratingsDf.count())  # "cachear"



CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 6.68 μs


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6212165207225505>, line 3
      1 get_ipython().run_line_magic('time', '')
      2 # Aplicar cache
----> 3 ratingsDf.cache()
      4 print("Tamaño: ", ratingsDf.count())

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:2163, in DataFrame.cache(self)
   2162 def cache(self) -> "DataFrame":
-> 2163     return self.persist()

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:2172, in DataFrame.persist(self, storageLevel)
   2167 def persist(
   2168     self,
   2169     storageLevel: StorageLevel = (StorageLevel.MEMORY_AND_DISK_DESER),
   2170 ) -> "DataFrame":
   2171     relation = self._plan.plan(self._session.client)
-> 2172     self._session.client._analyze(
   2173         method="persist", relation=relation, storage_level=storageLevel
   2174   

In [0]:
%time
# Probar con el cache
start = time.time()
ratingsDf.groupBy("userId").avg("rating").count()
print("Con cache:", time.time() - start, "segundos")



In [0]:
%time
# Limpiar caché
ratingsDf.unpersist()